In [446]:
from pymongo import MongoClient
import pymongo
import pprint as pp
from bs4 import BeautifulSoup
import requests
import pprint
from time import sleep
import sys
import re
import psycopg2
from datetime import date
from selenium import webdriver
import webkit_server
import dryscrape
from math import ceil

In [447]:
def get_last_page(url):
    try: 
        r1 = requests.get(url)
    except: 
        return "Failed to get request from url:", sys.exc_info()[0]
    soup_last_page = BeautifulSoup(r1.text).find(title='last page')
    if soup_last_page:
        last_page = int(re.findall('\w+',soup_last_page.text)[0])
        return last_page
    else: 
        raise Exception, "last_page title attribute not found in html"

In [448]:
def searched_users_to_db(mongodb_collection, url='https://boardgamegeek.com/users/page/', page_start=1, page_end=None, testing=False):
    '''
    The Boardgamegeek website displays only 25 users per page as of July, 2017. This function will step 1 unit from
    page_start to page_end and insert the returned webpage text into the supplied db.
    page_end will be appended to the end of the url string iteratively
    if page_end is set to None, last_page will be extracted automatically from the url, if possible
    '''
    
    page_end = get_last_page(url + str(page_start))  # the url only works if a page num is supplied
    
    for page_num in xrange(page_start, page_end+1):
        if testing==True:
            if page_num > 5:
                print "tested until page 5"
                break
        sleep(0.05)    
        url_ = url + str(page_num)
        r = requests.get(url_)
        print "working on page {} of {}. testing={}".format(page_num,page_end,testing)
        mongodb_collection.insert_one({'html':r.text, 'url':url_})
    

In [449]:
def extract_users_to_db(mongodb_collection_pages,mongodb_collection_users, \
                            root_users_url='https://boardgamegeek.com/collection', \
                            post_path='?rated=1&subtype=boardgame'):
    '''
    for each page in mongo_collection_pages, find all users on that page and insert 
    that users' info into mongodb_collection_users.
    Among the attributes saved for each user are: username, country (text) and
    ratings_url (which is concatenated like so: root_users_url + extracted_from_page_url + post_path)
    '''
    
    for page in mongodb_collection_pages.find():
        text = re.sub('[(\r)(\n)(\t)]',"",page['html'])
        soup = BeautifulSoup(text)
        users_in_page = soup.find_all('div','avatarblock js-avatar ')
        
        
        for user_field in users_in_page:
            
            user = user_field.find_next('div','username')
            
            username = user.text
            ratings_url = root_users_url + user.find_next('a').get('href') + post_path
            country = user_field.find("div","location js-location").text
            
            mongodb_collection_users.insert_one({'username' : username, \
                                                 'ratings_url' : ratings_url, \
                                                 'country' : country })

In [452]:
client = MongoClient()
db = client['bgg_data']
searched_pages = db['users_searched']
users_db = db['all_bgg_users']
games = db['games']
all_info = db['all_info']

In [580]:
searched_pages.remove()
searched_users_to_db(searched_pages, testing=True)

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


greater than 5


In [7]:
searched_pages.count()

5

In [575]:
users_db.remove()  #for testing

extract_users_to_db(searched_pages, users_db)

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


In [8]:
users_db.count()

125

In [9]:
users_db.find_one()

{u'_id': ObjectId('595d935e2966571815718d91'),
 u'country': u'United KingdomSouth Yorkshire.England',
 u'ratings_url': u'https://boardgamegeek.com/collection/user/zombiegod?rated=1&subtype=boardgame',
 u'username': u'zombiegod'}

In [10]:
type(users_db)

pymongo.collection.Collection

In [450]:
def extract_ratings_to_db(mongodb_collection_users,testing=True):
    i = 0
    for user in pymongo.cursor.Cursor(mongodb_collection_users):
        if not user.get('ratings'):
            r = requests.get(user.get('ratings_url'))
            user['ratings'] = r.text
            i += 1
            if i > 25:
                print "i is greater than 5, testing=True"
                break

In [22]:
%time extract_ratings_to_db(users_db,testing=True)

i is greater than 5, testing=True
CPU times: user 668 ms, sys: 16 ms, total: 684 ms
Wall time: 10.1 s


In [12]:
ratings_url = 'https://boardgamegeek.com/collection/user/zombiegod?rated=1&subtype=boardgame&ff=1'

In [13]:
ratings_r = requests.get(ratings_url)

In [14]:
soup1 = BeautifulSoup(ratings_r.text)

/home/aamir/.local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [15]:
game_names = []
for tag in soup1.find_all("td", { "class" : "collection_objectname" }):
    game_name = tag.find('a').text
    game_names.append(game_name)
    print game_name

10 Minute Heist: The Wizard's Tower
7 Wonders Duel
Abalone (Abalone Games Second edition 1999)
Abalone Travel
The Adventurers: The Temple of Chac (English first edition)
Age of Empires III: The Age of Discovery
Alhambra
The aMAZEing Labyrinth
Android: Netrunner
Animalia
Aqua Romana (Queen Multilingual Edition)
Arboretum
Arctic Scavengers (English second edition)
Arctic Scavengers: Base Game+HQ+Recon
Arena Maximus (European Edition)
Arkham Express
Arkham Horror: The Card Game
Army of Frogs (Tantrix UK Edition)
Asteroyds (Ystari french edition)
Asteroyds (Rio Grande Games English edition)
Automania
Automobiles (English first edition)
Ave Caesar (Ravensburger German Second Edition)
The Awful Green Things From Outer Space (Steve Jackson English Edition 2011)
Backgammon (Pavilion edition 1999)
Balloon Cup (Rio Grande English edition)
Bandu
Bandu
BANG! (daVinci English/Italian edition 2008)
BANG! The Dice Game (English/Italian second edition)
Batik (Gigamic stripe-box edition)
Battle Leader 

In [16]:
game_ratings = []
for tag in soup1.find_all("div", { "class" : "ratingtext" }):
    game_ratings.append(tag.text)
    print tag.text

6
6
5.9
5.9
6
5.5
6.9
6
7.5
6
7
8
8
7
6.2
5.5
7
7.5
7
7
8
9
6
5
7
4.8
8
8
7
8
4
4
6
8
10
5
6
5
5.9
3.9
6.9
7
4.8
6.7
7
6
9
6
8
9
9
7.8
4.5
5
8
6.9
4.5
3.9
5
5.8
6.5
6.9
8.9
8.9
7
7
6
5
7
4
4.5
5
6
6
8
5.5
6.5
5
5
6
3
8
6
7.9
4
8
8
4.9
4
4
8
8.5
5
5
6
8
6
8
3.5
7
6
7.7
4
6.5
6
5
8
9
1
7
6.5
6
5.5
5
6
7.5
8.9
6
8
7
8
10
8
7.9
4.5
4.5
5.5
8
7
10
7.9
5
8
9
7.5
7.5
1
8.9
7
8.9
10
5
2
7
5
5
6
5
5.9
7
4
5
8
1
8
7
8
8
3.8
8
6
7.9
6
8
8.5
10
6
5
5
5
5
4.9
7
5.5
7
7
7
6.6
3
3.5
3
6
6
6
5
5
4
8
10
7
10
10
8.5
6.8
6.9
7.9
5.5
7
6
6
7.5
7
8
6.9
7
5
6.5
6
7
4
7.8
3
5.5
5.5
6
6
7
4.5
8
6
6
7
4.5
7
6
3
2
10
5
6
10
7.5
7
5.9
8
7
8
10
6
4.8
5
7.5
6.9
6
5.8
5
6
8
3
3.7
7
5
5
6.6
4.5
7
5
5
8
8.9
7.9
6
8.5
8
10
7.8
5
6
6
5
5
3
5
9
6
5
9
6.8
10
9
7.5
7
4
7
7
6
6
6.9
9
5.7
6.7
5.9
5
4
7
7
6.5
5.9
5
7


In [17]:
print len(game_ratings)
print len(game_names)

300
300


In [453]:
def add_game_db(mongodb_collection_games,url=None, post_url=None, root_url=None, page_end=None, testing=True):

    if url==None:
        url = 'https://boardgamegeek.com/search/boardgame/page/'
    if post_url==None:
        post_url = '?sort=numvoters&advsearch=1&q=&include%5B' + \
            'designerid%5D=&include%5Bpublisherid%5D=&' + \
            'geekitemname=&range%5Byearpublished%5D%5B' + \
            'min%5D=&range%5Byearpublished%5D%5Bmax%5D=&range%5B' + \
            'minage%5D%5Bmax%5D=&range%5Bnumvoters%5D%5Bmin%5D=&range%5B' + \
            'numweights%5D%5Bmin%5D=&range%5Bminplayers%5D%5B' + \
            'max%5D=&range%5Bmaxplayers%5D%5Bmin%5D=&range%5B' + \
            'leastplaytime%5D%5Bmin%5D=&range%5Bplaytime%5D%5B' + \
            'max%5D=&floatrange%5Bavgrating%5D%5Bmin%5D=&floatrange%5B' + \
            'avgrating%5D%5Bmax%5D=&floatrange%5Bavgweight%5D%5B' + \
            'min%5D=&floatrange%5Bavgweight%5D%5Bmax%5D=&' + \
            'colfiltertype=&searchuser=&nosubtypes%5B0%5D=boardgameexpansion&playerrangetype=normal&' + \
            'B1=Submit&sortdir=desc'  #advanced filter with all options set to blank except
                                      #boardgame-expansions and output in descending order by num_voters
    if root_url==None:
        root_url = 'https://boardgamegeek.com'

    if page_end==None:
        page_end = get_last_page(url+'1'+post_url) #the url requires a page number. The choice of page 1 is arbitrary

    i=0  #only using for testing


    for page_num in xrange(1,page_end+1):
        if testing==True:
            i += 1
            if i > 10:
                return "testing=True ended the program at i greater than 2"
        url_ = url + str(page_num) + post_url
        sleep(0.025)
        r = requests.get(url_)
        print "working on page {} out of {}".format(page_num,page_end)
        soup = BeautifulSoup(r.text)
        table = soup.find('table','collection_table')
        rows = table.find_all(id='row_')
        for row in rows:
            game_url_name_tag = row.find('td','collection_objectname').find('a')
            game_name = game_url_name_tag.text
            game_url = root_url + game_url_name_tag.get('href')
            num_voters = row.find_all('td','collection_bggrating')[-1].text
            try:
                num_voters = int(num_voters)
            except ValueError:
                num_voters = 0
            except:
                num_voters = 0.0001  #leaving a numerical value to be safe but docstring shall have a comment about this
            game_id = int(re.findall(r'([0-9]+)/.*$',game_url)[0]) #extract game_id from url
            mongodb_collection_games.insert_one({'name':game_name,
                                                 'num_voters':num_voters,
                                                 'game_url':game_url,
                                                 'game_id':game_id})
    

In [457]:
add_game_db(games,testing=False)

working on page 1 out of 50
working on page 2 out of 50
working on page 3 out of 50
working on page 4 out of 50
working on page 5 out of 50
working on page 6 out of 50
working on page 7 out of 50
working on page 8 out of 50
working on page 9 out of 50
working on page 10 out of 50
working on page 11 out of 50
working on page 12 out of 50
working on page 13 out of 50
working on page 14 out of 50
working on page 15 out of 50
working on page 16 out of 50
working on page 17 out of 50
working on page 18 out of 50
working on page 19 out of 50
working on page 20 out of 50
working on page 21 out of 50
working on page 22 out of 50
working on page 23 out of 50
working on page 24 out of 50
working on page 25 out of 50
working on page 26 out of 50
working on page 27 out of 50
working on page 28 out of 50
working on page 29 out of 50
working on page 30 out of 50
working on page 31 out of 50
working on page 32 out of 50
working on page 33 out of 50
working on page 34 out of 50
working on page 35 out 

In [456]:
games.count()

0

In [455]:
games.remove()

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{u'n': 1600, u'ok': 1}

In [441]:
games.find()[2]

{u'_id': ObjectId('596304ba2966571299fdf785'),
 u'game_id': 30549,
 u'game_url': u'https://boardgamegeek.com/boardgame/30549/pandemic',
 u'name': u'Pandemic',
 u'num_voters': 64244}

In [444]:
def is_number(string):
    '''
    Returns 0 if text cannot be converted to float
    '''
    try:
        return float(string)
    except:
        return 0

In [445]:
def page_loading(sess_body_obj):
    soup1 = BeautifulSoup(sess_body_obj,'lxml')
    try:
        soup1.find_all('li','summary-item summary-rating-item')[0]
        return True
    except:
        return False

In [443]:
def visit_game_pg(mongodb_collection_games, mongodb_collection_all_info, search_date, testing=True):
    
    i=0
    for game in pymongo.cursor.Cursor(mongodb_collection_games):
        if not game.get('search_date') or game.get('search_date') < search_date:
            print "not searched yet, searching now...."
            url = game['game_url'] + '/ratings?rated=1&pageid=1' #filter out users that just commented but didn't rate
            print "game_url is {}".format(url)
            #https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=1
            server = webkit_server.Server()
            server_conn = webkit_server.ServerConnection(server=server)
            driver = dryscrape.driver.webkit.Driver(connection=server_conn)
            sess = dryscrape.Session(driver=driver)
            sess.visit(url)
            sess.wait_for(lambda: page_loading(sess.body()))
            source = sess.body()
            soup = BeautifulSoup(source,'lxml')
            sess.reset()
            last_page = soup.find_all('div','outline-item-description')[1].find('a').text
            last_page = int(re.sub("\W","",last_page))
            last_page = int(ceil(last_page*1./100))    #100 listings per page so 69145/100 --> 692, this page will have the last of it
            
            for page in xrange(1,last_page+1):
                url_page = game['game_url'] + ('/ratings?rated=1&pageid={}'.format(page))
                sess.visit(url_page)
                sess.wait_for(lambda: page_loading(sess.body()))
                print "extracting users & ratings from ratings page {} of total pages {}".format(page,last_page)
                source1 = sess.body()
                soup1 = BeautifulSoup(source1,'lxml')
                summary_item = soup1.find_all('li','summary-item summary-rating-item')
                sess.reset()
                print url_page
                
                for rating in summary_item:
                    player_rating = rating.find('div','summary-item-callout').text
                    player_name = rating.find('div','comment-header-title').find('a').text
                    mongodb_collection_all_info.insert_one({'game_name':game['name'],
                                                            'game_id':game['game_id'],
                                                            'player_name':player_name,
                                                            'player_rating':is_number(player_rating)})
                if testing==True:
                    i += 1
                    if i > 5:
                        return "Stopped after 5 pages because Testing = True, func visit_game_pg"    
            game['search_date'] = date.today()
            
    server.kill()
            
            

In [281]:
date(2017,7,1)

datetime.date(2017, 7, 1)

In [428]:
all_info.remove()

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{u'n': 300, u'ok': 1}

In [434]:
list(all_info.find())

[{u'_id': ObjectId('596328632966571299fdfc03'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'gauche',
  u'player_rating': 8.0},
 {u'_id': ObjectId('596328632966571299fdfc04'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'Korhil_NZ',
  u'player_rating': 3.0},
 {u'_id': ObjectId('596328632966571299fdfc05'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'711bgg',
  u'player_rating': 6.0},
 {u'_id': ObjectId('596328632966571299fdfc06'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'LyienneYyves',
  u'player_rating': 5.0},
 {u'_id': ObjectId('596328632966571299fdfc07'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'tyyyyygates',
  u'player_rating': 10.0},
 {u'_id': ObjectId('596328632966571299fdfc08'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_name': u'ciuppa',
  u'player_rating': 7.0},
 {u'_id': ObjectId('596328632966571299fdfc09'),
  u'game_id': 13,
  u'game_name': u'Catan',
  u'player_

In [430]:
visit_game_pg(games,all_info,date(2017,7,1),testing=True)

not searched yet, searching now....
game_url is https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=1
extracting users & ratings from ratings page 1 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=1
extracting users & ratings from ratings page 2 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=2
extracting users & ratings from ratings page 3 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=3
extracting users & ratings from ratings page 4 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=4
extracting users & ratings from ratings page 5 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=5
extracting users & ratings from ratings page 6 of total pages 692
https://boardgamegeek.com/boardgame/13/catan/ratings?rated=1&pageid=6


'Stopped after 5 pages because Testing = True, func visit_game_pg'

In [451]:
games.count()

1600